## LeNet
> - 激活函数：给卷积层后面加一个非线性变换，让整个网络不再是简单的线性变换叠加，从而能拟合更复杂的函数,不然两层卷积等价于一个大卷积
> 

In [ ]:
# import d2lzh as d2l
# import mxnet as mx
# from mxnet import autograd, gluon, init, nd
# from mxnet.gluon import loss as gloss, nn
# import time

# net = nn.Sequential()
# net.add(nn.Conv2D(channels=6, kernel_size=5, activation='sigmoid'),
#         nn.MaxPool2D(pool_size=2, strides=2),
#         nn.Conv2D(channels=16, kernel_size=5, activation='sigmoid'),
#         nn.MaxPool2D(pool_size=2, strides=2),
#         # Dense会默认将(批量大小, 通道, 高, 宽)形状的输入转换成
#         # (批量大小, 通道 * 高 * 宽)形状的输入
#         nn.Dense(120, activation='sigmoid'),
#         nn.Dense(84, activation='sigmoid'),
#         nn.Dense(10))
# X = nd.random.uniform(shape=(1, 1, 28, 28))
# net.initialize()
# for layer in net:
#     X = layer(X)
#     print(layer.name, 'output shape:\t', X.shape)

## AlexNet

In [ ]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import data as gdata, nn
import os
import sys

net = nn.Sequential()
# 使用较大的11 x 11窗口来捕获物体。同时使用步幅4来较大幅度减小输出高和宽。这里使用的输出通
# 道数比LeNet中的也要大很多
net.add(nn.Conv2D(96, kernel_size=11, strides=4, activation='relu'),
        nn.MaxPool2D(pool_size=3, strides=2),
        # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
        nn.Conv2D(256, kernel_size=5, padding=2, activation='relu'),
        nn.MaxPool2D(pool_size=3, strides=2),
        # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
        # 前两个卷积层后不使用池化层来减小输入的高和宽
        nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'),
        nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'),
        nn.Conv2D(256, kernel_size=3, padding=1, activation='relu'),
        nn.MaxPool2D(pool_size=3, strides=2),
        # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
        # 但在 图像分类 里，最后一步往往就是把空间压掉（Flatten + Dense 或 GlobalAvgPool），得到一个全局的语义表示。
        nn.Dense(4096, activation="relu"), nn.Dropout(0.5),
        nn.Dense(4096, activation="relu"), nn.Dropout(0.5),
        # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
        nn.Dense(10))

In [ ]:
# 本函数已保存在d2lzh包中方便以后使用
# root=os.path.join('~', '.mxnet', 'datasets', 'fashion-mnist')
# 得到字符串Windows："~\.mxnet\datasets\fashion-mnist"
# root = os.path.expanduser(root)
# root = "C:\\Users\\yangyuanhao\\.mxnet\\datasets\\fashion-mnist"
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'fashion-mnist')):
    root = os.path.expanduser(root)  # 展开用户路径'~'
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)
    mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
    mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4
    train_iter = gdata.DataLoader(
        mnist_train.transform_first(transformer), batch_size, shuffle=True,
        num_workers=num_workers)
    test_iter = gdata.DataLoader(
        mnist_test.transform_first(transformer), batch_size, shuffle=False,
        num_workers=num_workers)
    return train_iter, test_iter

batch_size = 128
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

In [ ]:
lr, num_epochs, ctx = 0.01, 5, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

# 经典 CNN 模型学习速记

> 学这些网络：**不要死记结构图**，只抓三点——「要解决什么问题」「用了什么新招」「这个招后来怎么被沿用」。

---

## 1. LeNet — CNN 起点

- **要解决的问题**：用端到端神经网络替代手工特征做小图像分类。
- **关键结构**：`卷积 + 池化 + 激活 + 全连接` 标准模板。
- **记忆要点**：
  - 局部感受野 + 权重共享 → 大幅减少参数。
  - 后续所有 CNN 都是在这套框架上加强演化。

---

## 2. AlexNet — 更深 + 更好训练

- **要解决的问题**：在大规模数据集（ImageNet）上训练更深网络。
- **关键新招**：
  - **ReLU**：非饱和激活，缓解梯度消失、加快训练。
  - **Dropout**：主要放在全连接层，减轻过拟合。
  - 更深网络 + 数据增强。
- **记忆要点**：
  - “深 + ReLU + Dropout + 大数据 + 数据增强” → 现代 CNN 训练基本套路的起点。

---

## 3. VGG — 小卷积核深堆叠

- **要解决的问题**：在控制参数量的前提下继续加深网络。
- **关键新招**：
  - 全用 **3×3 卷积** 堆叠：
    - 多个 3×3 ≈ 一个大卷积核，
    - 参数更少、非线性层更多。
  - 规则结构：`(3×3 conv × N) + max pool` 多次重复。
- **记忆要点**：
  - “小卷积核 + 深堆叠 + 规则 block” → 后续 ResNet / DenseNet 等 backbone 的基础设计风格。

---

## 4. NiN（Network in Network）— 卷积里的 MLP

- **要解决的问题**：增强**每个空间位置**上的特征表达能力，而不只是线性卷积。
- **关键新招**：
  - **1×1 卷积 = 通道维上的全连接（MLP）**：
    - 在每个像素位置对通道做非线性变换。
  - 多层 1×1 堆叠 = 每个位置一个小 MLP。
- **记忆要点**：
  - 1×1 卷积 = **通道混合 + 非线性**。  
  - 后续 Inception、通道注意力、残差中的“通道变换”都离不开它。

---

## 5. GoogLeNet / Inception — 多尺度 + 1×1 降维

- **要解决的问题**：同时利用多尺度卷积核，又不让参数和计算爆炸。
- **关键新招：Inception 模块**
  - 多分支并联：
    - 1×1 分支（基础特征）
    - 3×3 分支
    - 5×5 分支
    - 池化 + 1×1 分支  
    → 捕获多尺度特征。
  - 在 3×3、5×5 前加 **1×1 卷积**：
    - **降维**：先减通道数再做大卷积 → 减少参数与计算量。
    - **加非线性**：多一层激活，提高表达能力。
- **记忆要点**：
  - 核心关键词：**“多尺度并联 + 1×1 降维 + 特征拼接（concat）”**。
